## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,78.24,85,19,1.99,few clouds
1,1,Cape Town,ZA,-33.9258,18.4232,68.41,80,20,18.41,few clouds
2,2,Mataura,NZ,-46.1927,168.8643,66.47,41,56,1.52,broken clouds
3,3,Airai,TL,-8.9266,125.4092,66.42,75,42,2.73,scattered clouds
4,4,Chuy,UY,-33.6971,-53.4616,59.83,62,2,6.67,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
## 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,78.24,85,19,1.99,few clouds
1,1,Cape Town,ZA,-33.9258,18.4232,68.41,80,20,18.41,few clouds
2,2,Mataura,NZ,-46.1927,168.8643,66.47,41,56,1.52,broken clouds
3,3,Airai,TL,-8.9266,125.4092,66.42,75,42,2.73,scattered clouds
5,5,Gweta,BW,-20.1833,25.2333,76.17,41,43,8.99,scattered clouds
6,6,Hermanus,ZA,-34.4187,19.2345,65.80,81,61,18.45,broken clouds
7,7,Hue,VN,16.4667,107.6000,64.51,94,75,8.05,mist
9,9,Camacupa,AO,-12.0167,17.4833,60.42,95,94,2.42,light rain
11,11,Kulhudhuffushi,MV,6.6221,73.0700,79.86,77,100,4.29,overcast clouds
12,12,Charagua,BO,-19.8000,-63.2167,66.85,73,81,0.36,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID        276
City           276
Country        271
Lat            276
Lng            276
Max Temp       276
Humidity       276
Cloudiness     276
Wind Speed     276
Description    276
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hilo,US,19.7297,-155.0900,78.24,85,19,1.99,few clouds
1,1,Cape Town,ZA,-33.9258,18.4232,68.41,80,20,18.41,few clouds
2,2,Mataura,NZ,-46.1927,168.8643,66.47,41,56,1.52,broken clouds
3,3,Airai,TL,-8.9266,125.4092,66.42,75,42,2.73,scattered clouds
5,5,Gweta,BW,-20.1833,25.2333,76.17,41,43,8.99,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
700,700,Sinnamary,GF,5.3833,-52.9500,76.48,90,76,9.06,broken clouds
705,705,Maun,BW,-19.9833,23.4167,69.13,77,6,2.21,clear sky
706,706,Camopi,GF,3.1655,-52.3158,73.00,99,99,2.57,overcast clouds
708,708,Guatire,VE,10.4762,-66.5427,68.02,94,74,2.62,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,78.24,few clouds,19.7297,-155.0900,
1,Cape Town,ZA,68.41,few clouds,-33.9258,18.4232,
2,Mataura,NZ,66.47,broken clouds,-46.1927,168.8643,
3,Airai,TL,66.42,scattered clouds,-8.9266,125.4092,
5,Gweta,BW,76.17,scattered clouds,-20.1833,25.2333,
6,Hermanus,ZA,65.80,broken clouds,-34.4187,19.2345,
7,Hue,VN,64.51,mist,16.4667,107.6000,
9,Camacupa,AO,60.42,light rain,-12.0167,17.4833,
11,Kulhudhuffushi,MV,79.86,overcast clouds,6.6221,73.0700,
12,Charagua,BO,66.85,broken clouds,-19.8000,-63.2167,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Hilo,US,78.24,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,Cape Town,ZA,68.41,few clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
2,Mataura,NZ,66.47,broken clouds,-46.1927,168.8643,Ellie's Villa
5,Gweta,BW,76.17,scattered clouds,-20.1833,25.2333,Gweta Lodge
6,Hermanus,ZA,65.80,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))